# Putting it all together

Ovo je example program koji sadrži sve iz prijašnjih tutorijala

Kako ne radimo u direktoriju u kojem se nalazi robocup folder treba donje dvije komande izvršiti (samo jednom).

In [1]:
import sys
sys.path.append('../')

Uvezimo sve potrebno:

In [ ]:
import math
import time
import numpy as np
from robocup import field
from robocup import vision
from robocup import switches as s
from robocup import movement as mov
from robocup import commands as comm
from robocup import localization as loc
from multiprocessing import Process, Value
from robocup import bluetooth_communication as blu

robot_angle = 225

Napravimo dva različita procesa jedan za lokalizaciju drugi za procesuiranje slike:

In [2]:
xnum = Value('d', 0.0)
ynum = Value('d', 0.0)
anglenum = Value('d', 0.0)
distnum = Value('d', 0.0)
sizenum = Value('d', 0.0)
recnum = Value('d', 0.0)
erasenum = Value('d', 0.0)

p1 = Process(target=loc.localize, args=(robot_angle, xnum, ynum, False))
p2 = Process(target=vision.detect_the_ball, args=(anglenum, distnum, sizenum, recnum, erasenum, False))
p1.start()
p2.start()

640x480
640x480


Too many measurments in the input buffer: 574/500. Clearing buffer...
Too many measurments in the input buffer: 501/500. Clearing buffer...
Too many measurments in the input buffer: 615/500. Clearing buffer...


Ova četiri cella treba izvršiti s malim vremenskim odmakom (npr. 1s) služi za skupljanje statistike za obradu slike

In [3]:
erasenum.value = 1

In [4]:
erasenum.value = 0

In [5]:
recnum.value = 1

In [6]:
recnum.value = 0

## Napadač

#### ako vidimo loptu:

1. izračunamo kut pod kojim bi se trebali kretati da ga uhvatimo
2. promjenimo ga ukoliko ide prema crti
3. smanjimo brzinu ukoliko idemo prema crti 
4. smanjimo brzinu ukoliko je lopta blizu
5. 
ukoliko smo detektirali da smo van linije:
- vratimo se nazad

inače:
- idemo za balunom s izračunatim kutem


#### ako ne vidimo loptu:
- vratimo se u sredinu terena


In [7]:
def attacker():
    if seeball > 2:
        robot_speed = 0.3
        ba = mov.robot_angle_to_catch_ball(angle, x, y)
        _, ba = mov.flatten_angle_attacker(upper_dist=1200, x=x, y=y, ant=ba, speed=robot_speed)
        robot_speed, angle_out = mov.adaptive_speed_line_attacker(max_speed=0.4 , min_speed=0, upper_dist=800, x=x, y=y, ant=(ba)%360)
        robot_speed_ball = mov.adaptive_speed_ball(ba, dist, robot_speed, 0.3)
        min_speed = robot_speed if robot_speed_ball > robot_speed else robot_speed_ball
        if angle_out is None:
            comm.set_moving_angle(ba, min_speed)
        else:
            comm.set_moving_angle(angle_out, robot_speed)
    else:
        mov.move_to(x, y, field.x/2+100, field.y/2+350, 0.5)    

## Golman

#### ako vidimo loptu:

1. izračunamo kut pod kojim bi se trebali kretati da ga stanemo izmedu njega i gola
2. smanjimo brzinu ukoliko idemo prema crti (zamišljenoj koja se nalazi malo izvan jedanaesterca) 
3. ukoliko je lopta bliže od 20cm moramo je uhvatiti prema pravilima nakon 10s zato:
- zapamtimo vrijeme te ukoliko nakon 7s idalje stoji lopta bliže od 35cm (radi sigurnosti) idemo prema njoj


#### ako ne vidimo loptu:
- vratimo se u sredinu gola

In [8]:
def goalief(ball_angle, ball_size, ball_distance, goalie_last_time):
    robot_speed = 0.3
    if seeball > 2:
        dirr = math.sin(math.radians(ball_angle))
        ba = 0 if ball_angle<0 else 180
        robot_max_speed = 0.4*abs(dirr)
        robot_speed, angle_out = mov.adaptive_speed_line_goalie(max_speed=robot_max_speed , min_speed=0, upper_dist=200, x=x, y=y, ant=(ba)%360)
        
        
        if ball_distance > 350:
            goalie_last_time = time.time()  
        if time.time() - goalie_last_time  > 7:
            attacker()
            return goalie_last_time
        min_speed = robot_speed
        if angle_out is None:
            comm.set_moving_angle(ba, min_speed)
        else:
            comm.set_moving_angle(angle_out, robot_speed)
    else:
        mov.move_to(x, y, field.x/2, 520, 0.7)
        goalie_last_time = time.time()
    return goalie_last_time    
        
        

## Glavna Petlja
Prvo čekamo da se stisne prekidač0 pa stavimo orijentaciju i čekamo ponovno.

1. skupimo updateane vrijednosti koje su nam procesi p1 i p2 dali
2. seeball je varijabla koja sprečava da naglo promjenimo sve ukoliko jedan frame nismo vidjeli loptu, ona služi da se izbjegne trzanje koje bi nastalo ako tren vidimo loptu tren ne
3. ukoliko neku koordinatu nismo detektirali koristit cemo zadnju
4. gledamo prekidač2:
- ukoliko je u stanju 1 ---> golman
- ukoliko je u stanju 0 ---> napadač
. ako je lopta blizu udarimo kickerom
 



In [ ]:
while s.state(0):
    a = 0
comm.set_orientation()
while not s.state(0):
    a = 0
comm.start()
x = field.x / 2 + 0.01
y = field.y / 2 + 0.012
seeball = 0
robot_speed = 0.1
goalie = False
goalie_last_time = 0

while True:
    xc = xnum.value
    yc = ynum.value
    angle = anglenum.value
    ball_size = sizenum.value
    dist = distnum.value   

    seeball += 1 if ball_size > 200 else -3 
    if seeball > 10:
        seeball = 10
    if seeball < -1:
        seeball = -1
        
    if (xc != -1):
        x = xc
    if (yc != -1): 
        y = yc
    if not s.state(2):   
        goalie_last_time = goalief(ball_angle = angle, ball_size = ball_size, ball_distance = dist, goalie_last_time=goalie_last_time)
    else:
        attacker()
    if angle > -10 and angle<10 and dist < 155 and dist > 135 and ball_size > 1300 and ball_size < 1600:
        comm.kick()
        time.sleep(0.005)
    time.sleep(0.005)
    s.check_pause()